In [1]:
pip install langchain pypdf


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 20.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 269.8/269.8 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 2.7 MB/s eta 0:00:00


In [2]:
# PDF Loaders
from langchain.document_loaders import UnstructuredPDFLoader, OnlinePDFLoader, PyPDFLoader

from langchain.text_splitter import RecursiveCharacterTextSplitter
import os

In [3]:
loader = PyPDFLoader("ebook.pdf")

In [4]:
data = loader.load()

In [5]:
# Note: If you're using PyPDFLoader then it will split by page for you already
print (f'You have {len(data)} document(s) in your data')
print (f'There are {len(data[30].page_content)} characters in your document')

You have 898 document(s) in your data
There are 1424 characters in your document


In [6]:
pip install tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 32.6 MB/s eta 0:00:00


### Chunk your data up into smaller documents

In [7]:
# Note: If you're using PyPDFLoader then we'll be splitting for the 2nd time.
# This is optional, test out on your own data.

text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=0)
texts = text_splitter.split_documents(data)

In [8]:
print (f'Now you have {len(texts)} documents')

Now you have 995 documents


In [11]:
pip install pinecone-client

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.1/179.1 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.3/300.3 kB 23.9 MB/s eta 0:00:00


### Create embeddings of your documents to get ready for semantic search

In [12]:
from langchain.vectorstores import Chroma, Pinecone
from langchain.embeddings.openai import OpenAIEmbeddings
import pinecone

/usr/local/lib/python3.10/dist-packages/pinecone/index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [15]:
pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 3.4 MB/s eta 0:00:00


In [13]:
# Check to see if there is an environment variable with you API keys, if not, use what you put below
OPENAI_API_KEY = os.environ.get('OPENAI_API_KEY', 'your_key')

PINECONE_API_KEY = os.environ.get('PINECONE_API_KEY', 'your_key')
PINECONE_API_ENV = os.environ.get('PINECONE_API_ENV', 'asia-southeast1-gcp-free') # You may need to switch with your env

In [16]:
embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)

In [17]:
# initialize pinecone
pinecone.init(
    api_key=PINECONE_API_KEY,  # find at app.pinecone.io
    environment=PINECONE_API_ENV  # next to api key in console
)
index = "astro" # put in the name of your pinecone index here

In [18]:
docsearch = Pinecone.from_texts([t.page_content for t in texts], embeddings, index_name=index)

In [19]:
query = "What does Rahu do in the second house?"
docs = docsearch.similarity_search(query)

In [20]:
# Here's an example of the first document that was returned
print(docs[0].page_content[:450])

Rahu in the 2nd House aspects the 6th, 8th,and 10th House.  Rahu  
in the 2nd House brings a very unusual family  upbringing.  One 
wants to be the head of the family  and know everything; from  
lineage to family assets. This placement  brings obsession with 
wealth, gems, family  assets, and savings. One is always striving to 
save as much  as possible, but they may not be able to until their 
30's. Rahu wants to have the most  perfect family  


### Query those docs to get your answer back

In [21]:
from langchain.llms import OpenAI
from langchain.chains.question_answering import load_qa_chain
from langchain.chains import ConversationChain


In [22]:
llm = OpenAI(temperature=0, model_name='gpt-3.5-turbo-0613', openai_api_key=OPENAI_API_KEY)

/usr/local/lib/python3.10/dist-packages/langchain/llms/openai.py:200: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/langchain/llms/openai.py:801: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


In [30]:
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory
from langchain.memory import ConversationTokenBufferMemory

template = """You are a famous Vedic astrologer making predictions based on your clients' horoscopes. Synthesize the knowledge of GPT with the following extracted parts of a document and a question, then create a final answer.

{context}
{chat_history}
Human: {human_input}
Chatbot:"""

prompt = PromptTemplate(
    input_variables=["chat_history", "human_input", "context"], template=template, k=5
)
memory = ConversationTokenBufferMemory(llm=llm, memory_key="chat_history", input_key="human_input", verbose=True)
chain = load_qa_chain(llm, chain_type='stuff', memory=memory, prompt=prompt)



In [31]:
query = """What will be the best period (age wise) for this individual? Analyze all his below details:
Sun is in Cancer which is a Friendly sign.
Sun is lord of 5th house and situated in 4th House.
Sun fully aspects 10th house.
Sun is aspected by SATURN , KETU .Sun is in Ashlesha nakshatra.
Moon is in Sagittarius which is a Neutral sign.
Moon is lord of 4th house and situated in 9th House.
Moon fully aspects 3rd house.
Moon is aspected by SATURN . Moon is in	Purvashadha.
Mars is in Scorpion which is a Own sign.
Mars is lord of 1st, 8th house and situated in 8th House.
Mars fully aspects 11th, 2nd, 3rd house.
Mars is aspected by RAHU . Mars is in	Vishakha.
Mercury is in Leo which is a Friendly sign.
Mercury is lord of 3rd,6th house and situated in 5th House.
Mercury fully aspects 11th house.
Mercury is aspected by JUPITER. Mercury is in Purvaphalgini.
Jupiter is in Sagittarius which is a Own sign.
Jupiter is lord of 9th, 12th house and situated in 9th House.
Jupiter fully aspects 1st, 3rd, 5th house.
Jupiter is aspected by SATURN . Jupiter is in Mula.
Venus is in Leo which is a Enemy sign.
Venus is lord of 2nd, 7th house and situated in 5th House.
Venus fully aspects 11th house.
Venus is aspected by JUPITER . Venus is in Magha.
Saturn is in Libra which is a Exalted sign.
Saturn is lord of 10th, 11th house and situated in 7th House.
Saturn fully aspects 9th, 1st, 4th House .Saturn is in	Swati.
Rahu is in Taurus. Rahu is situated in 2nd House.
Rahu fully aspects 6th, 8th, 10th house.
Rahu is aspected by MARS, KETU. Rahu	is in Krittika.
Ketu is in Scorpion. Ketu is situated in 8th House.
Ketu fully aspects 12th, 2nd, 4th house.
Ketu is aspected by RAHU. Ketu	is in Anuradha.
"""
docs = docsearch.similarity_search(query)
chain({"input_documents": docs, "human_input": query}, return_only_outputs=True)

{'output_text': 'Based on the details provided, the best period for this individual would be after the age of 32. The Sun, which is the lord of the 5th house, is situated in the 4th house, indicating that the individual will experience some challenges in their home life until the age of 35. However, after the age of 32, they can expect a good career and success. Additionally, the Sun fully aspects the 10th house, which is related to career and status, further indicating potential success in this area.'}

In [33]:
print(chain.memory.buffer)

[HumanMessage(content='What will be the best period (age wise) for this individual? Analyze all his below details:\nSun is in Cancer which is a Friendly sign.\nSun is lord of 5th house and situated in 4th House.\nSun fully aspects 10th house.\nSun is aspected by SATURN , KETU .Sun is in Ashlesha nakshatra.\nMoon is in Sagittarius which is a Neutral sign.\nMoon is lord of 4th house and situated in 9th House.\nMoon fully aspects 3rd house.\nMoon is aspected by SATURN . Moon is in\tPurvashadha.\nMars is in Scorpion which is a Own sign.\nMars is lord of 1st, 8th house and situated in 8th House.\nMars fully aspects 11th, 2nd, 3rd house.\nMars is aspected by RAHU . Mars is in\tVishakha.\nMercury is in Leo which is a Friendly sign.\nMercury is lord of 3rd,6th house and situated in 5th House.\nMercury fully aspects 11th house.\nMercury is aspected by JUPITER. Mercury is in Purvaphalgini.\nJupiter is in Sagittarius which is a Own sign.\nJupiter is lord of 9th, 12th house and situated in 9th Hou

In [32]:
query = """How will Saturn dasha go for this individual?"""
docs = docsearch.similarity_search(query)
chain({"input_documents": docs, "human_input": query}, return_only_outputs=True)

{'output_text': "Saturn dasha will be generally favorable for this individual. Saturn is exalted in Libra and is the lord of the 10th and 11th houses, which are related to career and gains. This indicates that during the Saturn dasha, the individual may experience growth and success in their career, as well as financial gains. However, it is important to consider the placement of Saturn in the individual's chart and its aspects with other planets. If Saturn is in conjunction or aspect with malefic planets, it may create some challenges or obstacles during the dasha period. Overall, the Saturn dasha has the potential to bring positive results, especially in terms of career and financial stability."}

In [34]:
query = """Will this individual settle far from this birthplace? If so, why?"""
docs = docsearch.similarity_search(query)
chain({"input_documents": docs, "human_input": query}, return_only_outputs=True)

{'output_text': "Yes, according to the individual's birth chart, there are indications of settling far from their birthplace. The placement of planets such as Rahu, Ketu, Jupiter, and the lords of the 2nd, 4th, and 7th houses suggest foreign settlements or living away from their homeland. The 4th house represents land and mother's home, and when planets like Jupiter, Rahu, or Ketu are present in the 4th house, they indicate foreign settlements. Additionally, the 7th house, which is 4th from the 4th house, represents a home away from home. Therefore, based on these astrological factors, it is likely that this individual will settle far from their birthplace."}

In [35]:
query = """If you were the individual's friend, how will you describe him?"""
docs = docsearch.similarity_search(query)
chain({"input_documents": docs, "human_input": query}, return_only_outputs=True)

{'output_text': "If I were the individual's friend, I would describe him as someone who has a strong sense of ambition and drive. With the Sun in Cancer, he is likely to be a caring and nurturing individual, but also someone who is determined to achieve his goals. The placement of Mars in the 8th house indicates that he may have a fearless and intense nature, which can make him a strong leader or someone who excels in challenging situations. Additionally, the presence of Jupiter in the 9th house suggests that he may have a philosophical and expansive mindset, always seeking knowledge and growth. Overall, he is likely to be a dynamic and inspiring individual who is focused on achieving success in his chosen path."}

In [37]:
query = """Can this individual succeed in the domain of AI?"""
docs = docsearch.similarity_search(query)
chain({"input_documents": docs, "human_input": query}, return_only_outputs=True)

{'output_text': "Yes, Rahu in the 2nd house aspects the 10th house, which represents unconventional thinking and technological advancements, can indicate potential success in the domain of AI. Rahu is associated with innovation and breaking taboos, which are important qualities in the field of AI. Additionally, the aspect of Mars on Mercury further enhances the individual's analytical and technical abilities, which are crucial in AI. However, it is important to consider other factors such as education, training, and experience to determine the individual's full potential for success in the specific domain of AI."}